## 통합 네트워크 모델
- #### 1. CNN model  
- #### 2. LANDMARK_FCN model
- #### 3. CNN + FCN

- ### 0. 데이터 구축

In [1]:
# import zipfile

In [2]:
# path_fer = 'C:/Users/SAMSUNG/humandata/fer2013.zip'
# zip_object_fer = zipfile.ZipFile(file =path_fer , mode = 'r')
# zip_object_fer.extractall('./')
# zip_object_fer.close()

In [3]:
import tensorflow as tf
import numpy as np
import shutil
import matplotlib.pyplot as plt
from PIL import Image
import os
import cv2 ,dlib,math
import glob
import pandas 

### DATA  SET 
- ### IMAGE
- ### LANDMARK

#### (1) DATA SET 구축

In [4]:
Datas = 'C:/Users/SAMSUNG/humandataho/img'
Dataset = []
Folders = ['train', 'test']
CATEGORIES = ['angry','fear','happy','neutral','sad','surprise']

In [5]:
Datas = 'C:/Users/SAMSUNG/humandataho/img'
Train_set, Test_set = [], []
Folders = ['train', 'test']
Categories = ['angry','fear','happy','neutral','sad','surprise']

In [6]:
predictor = dlib.shape_predictor("C:/Users/SAMSUNG/humandata/shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

In [7]:
X_train_landmarks, X_train_images, y_train = [], [], []
X_test_landmarks, X_test_images, y_test = [], [], []

In [8]:
def getLandmarks(img):
    
    landmarks = []
    
    dets = detector(img,1)
    
    for k, d in enumerate(dets): 
        shape = predictor(img, d) #shape: 얼굴 랜드마크 추출 

        for i in range(shape.num_parts): #총 68개
            shape_point = shape.part(i)
            landmarks.append([shape_point.x, shape_point.y])
    
    return landmarks

In [9]:
for folder in Folders:
    for category in Categories:
        count = 0
        print("작동중.")
        PATH = os.path.join(Datas, folder)
        PATH = os.path.join(PATH, category)
        for el in os.listdir(PATH):
            img = cv2.imread(os.path.join(PATH,el),0)            
            
            try:                
                if (category == "angry"):
                     a = [1,0,0,0,0,0]
                    #a = 1
                if (category == "fear"):
                     a = [0,1,0,0,0,0]
                    #a = 2
                if (category == "happy"):
                     a = [0,0,1,0,0,0]
                    #a = 3
                if (category == "neutral"):
                     a = [0,0,0,1,0,0]
                    #a = 4
                if (category == "sad"):
                     a = [0,0,0,0,1,0]
                    #a = 5
                if(category == "surprise"):
                     a = [0,0,0,0,0,1]
                    #a = 6
                
                landmarks = getLandmarks(img)
                
                modified_img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                
#                 modified_img = cv2.resize(modified_img,(224,224))
                
                ## add features
                
                x_coordinates = [landmark[0] for landmark in landmarks]
                y_coordinates = [landmark[1] for landmark in landmarks]
                
                if len(x_coordinates) == 0:
                    continue
                
                left_eye_center_x = np.mean(x_coordinates[36:42])
                left_eye_center_y = np.mean(y_coordinates[36:42])
                
                right_eye_center_x = np.mean(x_coordinates[42:48])
                right_eye_center_y = np.mean(y_coordinates[42:48])

                nose_center_x = np.mean(x_coordinates[27:36])
                nose_center_y = np.mean(y_coordinates[27:36])

                mouth_center_x = np.mean(x_coordinates[48:68])
                mouth_center_y = np.mean(y_coordinates[48:68])
                
                # Calculate relative distances between centers
                eye_to_nose_x = abs(left_eye_center_x - nose_center_x)
                eye_to_nose_y = abs(left_eye_center_y - nose_center_y)
                
                eye_to_mouth_x = abs(left_eye_center_x - mouth_center_x)
                eye_to_mouth_y = abs(left_eye_center_y - mouth_center_y)
                
                nose_to_mouth_x = abs(nose_center_x - mouth_center_x)
                nose_to_mouth_y = abs(nose_center_y - mouth_center_y)
                
                landmarks.append([left_eye_center_x, left_eye_center_y])
                landmarks.append([right_eye_center_x, right_eye_center_y])
                landmarks.append([nose_center_x, nose_center_y])
                landmarks.append([mouth_center_x, mouth_center_y])
                landmarks.append([eye_to_nose_x, eye_to_nose_y])
                landmarks.append([eye_to_mouth_x, eye_to_mouth_y])
                landmarks.append([nose_to_mouth_x, nose_to_mouth_y])            
                
                
                if(len(landmarks) != 0):
                    if (folder == 'train'):
                        X_train_images.append(modified_img)
                        X_train_landmarks.append(landmarks)
                        y_train.append(a)
                    if (folder == 'test'):
                        X_test_images.append(modified_img)
                        X_test_landmarks.append(landmarks)
                        y_test.append(a)
            except Exception as e:
                print(e)

작동중.
작동중.
작동중.
작동중.
작동중.
작동중.
작동중.
작동중.
작동중.
작동중.
작동중.
작동중.


In [10]:
X_train_landmarks = np.array(X_train_landmarks).astype('int8') 
X_train_images = np.array(X_train_images).astype('int8')
y_train = np.array(y_train)
X_test_landmarks = np.array(X_test_landmarks).astype('int8')
X_test_images = np.array(X_test_images).astype('int8')
y_test = np.array(y_test)

print(X_train_landmarks.shape)
print(X_train_images.shape)
print(X_test_landmarks.shape)
print(X_test_images.shape)

(12465, 75, 2)
(12465, 48, 48, 3)
(3625, 75, 2)
(3625, 48, 48, 3)


In [11]:
num_train_data = len(X_train_landmarks)
num_test_data = len(X_test_landmarks)

index_train = np.arange(num_train_data)
index_test = np.arange(num_test_data)

np.random.shuffle(index_train)
np.random.shuffle(index_test)

X_train_landmarks = X_train_landmarks[index_train]
X_train_images = X_train_images[index_train]
y_train = y_train[index_train]

X_test_landmarks = X_test_landmarks[index_test]
X_test_images = X_test_images[index_test]
y_test = y_test[index_test]

In [12]:
X_train_landmarks = np.array(X_train_landmarks).astype('float32') / 48
X_train_images = np.array(X_train_images).astype('float32').reshape(-1,48,48,3)/ 255

y_train = np.array(y_train)
X_test_landmarks = np.array(X_test_landmarks).astype('float32') / 48
X_test_images = np.array(X_test_images).astype('float32').reshape(-1,48,48,3) /255

y_test = np.array(y_test)

In [13]:


# folder1 = 'C:/Users/SAMSUNG/humandataho/img/train/surprise'   #surprise
# folder2 = 'C:/Users/SAMSUNG/humandataho/img/train/sad'  #sad

# folder3 = 'C:/Users/SAMSUNG/humandataho/img/train/neutral'    #neutral

# folder4 = 'C:/Users/SAMSUNG/humandataho/img/train/happy'    # happy

# folder5 = 'C:/Users/SAMSUNG/humandataho/img/train/fear'    # fear

# folder6 = 'C:/Users/SAMSUNG/humandataho/img/train/angry'     # angry


# extension = '.jpg'

# folder1 = glob.glob(folder1 + '/*.jpg')
# folder2 = glob.glob(folder2 + '/*.jpg')
# folder3 = glob.glob(folder3 + '/*.jpg')
# folder4 = glob.glob(folder4 + '/*.jpg')
# folder5 = glob.glob(folder5 + '/*.jpg')
# folder6 = glob.glob(folder6 + '/*.jpg')
# X_train = []
# y_train = []
# for file_name in folder1:
#     image = cv2.imread(file_name)
#     X_train.append(image)
#     y_train.append(np.array([1,0,0,0,0,0]))


# for file_name in folder2:
#     image = cv2.imread(file_name)
#     X_train.append(image)
#     y_train.append(np.array([0,1,0,0,0,0]))

# for file_name in folder3:
#     image = cv2.imread(file_name)
#     X_train.append(image)
#     y_train.append(np.array([0,0,1,0,0,0]))

# for file_name in folder4:
#     image = cv2.imread(file_name)
#     X_train.append(image)
#     y_train.append(np.array([0,0,0,1,0,0]))

# for file_name in folder5:
#     image = cv2.imread(file_name)
#     X_train.append(image)
#     y_train.append(np.array([0,0,0,0,1,0]))
    
# for file_name in folder6:
#     image = cv2.imread(file_name)
#     X_train.append(image)
#     y_train.append(np.array([0,0,0,0,0,1]))
    
# X_train = np.array(X_train)
# y_train = np.array(y_train)
# X_train = X_train.astype('float32') / 255

#### (2) test_data_set 구축  , test_label 구축

In [14]:
# folder10 = 'C:/Users/SAMSUNG/humandataho/img/test/surprise'     #surprise
# folder20 = 'C:/Users/SAMSUNG/humandataho/img/test/sad'    # sad

# folder30 = 'C:/Users/SAMSUNG/humandataho/img/test/neutral'   # neutral

# folder40 = 'C:/Users/SAMSUNG/humandataho/img/test/happy'  # happy

# folder50 = 'C:/Users/SAMSUNG/humandataho/img/test/fear' #fear

# folder60 = 'C:/Users/SAMSUNG/humandataho/img/test/angry'   # angry




# extension = '.jpg'

# folder10 = glob.glob(folder10 + '/*.jpg')
# folder20 = glob.glob(folder20 + '/*.jpg')
# folder30 = glob.glob(folder30 + '/*.jpg')
# folder40 = glob.glob(folder40 + '/*.jpg')
# folder50 = glob.glob(folder50 + '/*.jpg')
# folder60 = glob.glob(folder60 + '/*.jpg')
# X_test = []
# y_test = []
# for file_name_t in folder10:
#     image = cv2.imread(file_name)
   
#     X_test.append(image)
#     y_test.append(np.array([1,0,0,0,0,0]))


# for file_name_t in folder20:
#     image = cv2.imread(file_name_t)
#     X_test.append(image)
#     y_test.append(np.array([0,1,0,0,0,0]))

# for file_name_t in folder30:
#     image = cv2.imread(file_name_t)
#     X_test.append(image)
#     y_test.append(np.array([0,0,1,0,0,0]))

# for file_name_t in folder40:
#     image = cv2.imread(file_name_t)
#     X_test.append(image)
#     y_test.append(np.array([0,0,0,1,0,0]))

# for file_name_t in folder50:
#     image = cv2.imread(file_name_t)
#     X_test.append(image)
#     y_test.append(np.array([0,0,0,0,1,0]))
    
# for file_name_t in folder60:
#     image = cv2.imread(file_name_t)
#     X_test.append(image)
#     y_test.append(np.array([0,0,0,0,0,1]))
    
# X_test = np.array(X_test)
# y_test = np.array(y_test)
# X_test = X_test.astype('float32') / 255

#### train , test data 인덱스 맞춰주면서 섞어주기

In [15]:
# num_data = len(X_train)

# index_arr = np.arange(num_data)

# np.random.shuffle(index_arr)

# X_train = X_train[index_arr]

# y_train = y_train[index_arr]

# index_arr
# print(X_train.shape)
# print(y_train.shape)

In [16]:

# num_datas = len(X_test)

# index_arrs = np.arange(num_datas)

# np.random.shuffle(index_arrs)

# X_test = X_test[index_arrs]

# y_test = y_test[index_arrs]


# print(X_test.shape)
# print(y_test.shape)

### 1. CNN 모델  - 기본

In [17]:

# from tensorflow import keras
# from tensorflow.keras import layers
# inputs = keras.Input(shape=(48, 48, 1))
# x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
# x = layers.MaxPooling2D(pool_size=2)(x)
# x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
# x = layers.Flatten()(x)
# outputs = layers.Dense(6, activation="softmax")(x)

# model = keras.Model(inputs=inputs, outputs=outputs)

In [18]:
# model.summary()

### 1.1 model compile, fit

In [19]:
# model.compile(optimizer="adam",
#     loss="categorical_crossentropy",
#     metrics=["accuracy"])
# history = model.fit(X_train_images, y_train, epochs=20, batch_size=64, validation_split=0.2, callbacks=callbacks_list)

In [20]:
# test_loss, test_acc = model.evaluate(X_test_images, y_test)
# print(f"테스트 정확도: {test_acc:.3f}")

## FCN Code with add feature

In [45]:
from tensorflow import keras
from tensorflow.keras import layers

In [49]:
callbacks_list = [
keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=10,
)]

In [50]:
inputs = keras.Input(shape=(75,2), name="landmarks")
features = layers.Dense(100, activation="relu")(inputs)
features = layers.Dense(150, activation="relu")(features)
features = layers.Dropout(0.5)(features)
# features = layers.Dense(150, activation="relu")(features)
features = layers.Dense(100, activation="relu")(features)
features = layers.Flatten()(features)
outputs = layers.Dense(6, activation="softmax")(features)
models = keras.Model(inputs=inputs, outputs=outputs)

In [51]:
models.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 landmarks (InputLayer)      [(None, 75, 2)]           0         
                                                                 
 dense_6 (Dense)             (None, 75, 100)           300       
                                                                 
 dense_7 (Dense)             (None, 75, 150)           15150     
                                                                 
 dropout_2 (Dropout)         (None, 75, 150)           0         
                                                                 
 dense_8 (Dense)             (None, 75, 100)           15100     
                                                                 
 flatten_2 (Flatten)         (None, 7500)              0         
                                                                 
 dense_9 (Dense)             (None, 6)                 4500

In [52]:
models.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [53]:
models.fit(
    X_train_landmarks,
    y_train,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/100
156/156 [==============================] - 3s 15ms/step - loss: 1.7847 - accuracy: 0.1963 - val_loss: 1.7714 - val_accuracy: 0.2475
Epoch 2/100
156/156 [==============================] - 2s 13ms/step - loss: 1.7072 - accuracy: 0.2863 - val_loss: 1.5721 - val_accuracy: 0.3799
Epoch 3/100
156/156 [==============================] - 2s 14ms/step - loss: 1.5635 - accuracy: 0.3664 - val_loss: 1.4999 - val_accuracy: 0.3967
Epoch 4/100
156/156 [==============================] - 2s 16ms/step - loss: 1.5028 - accuracy: 0.4028 - val_loss: 1.4700 - val_accuracy: 0.4140
Epoch 5/100
156/156 [==============================] - 2s 15ms/step - loss: 1.4711 - accuracy: 0.4186 - val_loss: 1.4160 - val_accuracy: 0.4533
Epoch 6/100
156/156 [==============================] - 2s 13ms/step - loss: 1.4544 - accuracy: 0.4188 - val_loss: 1.4541 - val_accuracy: 0.4332
Epoch 7/100
156/156 [==============================] - 2s 13ms/step - loss: 1.4366 - accuracy: 0.4300 - val_loss: 1.4054 - val_accuracy:

In [54]:
models.evaluate(X_test_landmarks, y_test)

114/114 [==============================] - 1s 5ms/step - loss: 1.3228 - accuracy: 0.4921


[1.3228455781936646, 0.49213793873786926]

### 1. CNN 모델  - VGGNET

In [28]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
from keras import models
from keras import layers
from keras import optimizers
import keras.backend as K

In [29]:
from keras.applications import VGG16

In [30]:
conv_layers = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
conv_layers.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [31]:
for layer in conv_layers.layers:
    layer.trainable = False


In [32]:
model = models.Sequential()
model.add(conv_layers)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='softmax'))

In [33]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 1024)              525312    
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_5 (Dense)             (None, 6)                 6150      
                                                                 
Total params: 15,246,150
Trainable params: 531,462
Non-trainable params: 14,714,688
_________________________________________________________________


In [34]:
# 모델 컴파일
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
# 모델 학습
model.fit(X_train_images, y_train, batch_size=64, epochs=100, validation_split=0.2,callbacks=callbacks_list)

Epoch 1/100
156/156 [==============================] - 69s 438ms/step - loss: 1.7312 - accuracy: 0.2826 - val_loss: 1.6347 - val_accuracy: 0.3165
Epoch 2/100
156/156 [==============================] - 66s 424ms/step - loss: 1.6303 - accuracy: 0.3327 - val_loss: 1.6312 - val_accuracy: 0.3281
Epoch 3/100
156/156 [==============================] - 66s 424ms/step - loss: 1.6094 - accuracy: 0.3457 - val_loss: 1.6052 - val_accuracy: 0.3365
Epoch 4/100
156/156 [==============================] - 66s 425ms/step - loss: 1.5757 - accuracy: 0.3669 - val_loss: 1.5959 - val_accuracy: 0.3458
Epoch 5/100
156/156 [==============================] - 77s 496ms/step - loss: 1.5615 - accuracy: 0.3729 - val_loss: 1.5950 - val_accuracy: 0.3422
Epoch 6/100
156/156 [==============================] - 75s 481ms/step - loss: 1.5419 - accuracy: 0.3815 - val_loss: 1.5978 - val_accuracy: 0.3430
Epoch 7/100
156/156 [==============================] - 66s 424ms/step - loss: 1.5247 - accuracy: 0.3908 - val_loss: 1.5932 -

In [36]:
test_loss, test_acc = model.evaluate(X_test_images, y_test)
print(f"테스트 정확도: {test_acc:.3f}")

114/114 [==============================] - 24s 212ms/step - loss: 2.1585 - accuracy: 0.3961
테스트 정확도: 0.396


In [ ]:
model.save("VGG_model.h5")

### 3. CNN모델 + landmark_FCN 모델

In [42]:
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model

In [55]:
cnn_predictions = model.predict( X_test_images)



114/114 [==============================] - 22s 193ms/step


In [56]:
fcn_predictions = models.predict(X_test_landmarks)

114/114 [==============================] - 3s 26ms/step


In [66]:
# add방법 - cnn(기본) + FCN

cnn_predictions = cnn_predictions * 0.0001
fcn_predictions = fcn_predictions * 0.9999

total_prediction = cnn_predictions + fcn_predictions

temp_add = np.argmax(total_prediction, axis=1)

temp_add = keras.utils.to_categorical(temp_add, num_classes=6).astype('int8')

        
accuracy_add = accuracy_score(y_test, temp_add)

accuracy_add

0.4921379310344828

In [57]:
# # add방법 - cnn(VGGNET) + FCN

# cnn_predictions = VGG_predictions * 0.0001
# fcn_predictions = fcn_predictions * 0.9999

# total_prediction = VGG_predictions + fcn_predictions

# temp_add = np.argmax(total_prediction, axis=1)

# temp_add = keras.utils.to_categorical(temp_add, num_classes=6).astype('int8')

        
# accuracy_add = accuracy_score(y_test, temp_add)

# accuracy_add

NameError: name 'VGG_predictions' is not defined